#**Aivle 스쿨 지원 질문, 답변 챗봇 만들기**
# 단계2 : 전처리

## 0.미션

* 1) 데이터 분할
    * 학습/평가 데이터를 분리합니다.

* 2) 전처리
    * 학습을 위한 전처리 : 자연어 처리를 위한 전처리를 수행합니다.
        * 맞춤법 검사 라이브러리를 활용하여 올바른 문장으로 변환합니다.
        * 텍스트 데이터 증강 라이브러리를 활용하여 문장의 다양성을 높입니다.
    * 전처리 결과를 저장합니다.

## 1.환경준비

* 세부 요구사항
    - 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다.
    - 필요하다고 판단되는 라이브러리를 추가하세요.
    - 구글드라이브를 통해 데이터를 로딩합니다.

### (1) 라이브러리 불러오기

In [1]:
import pandas as pd
import numpy as np

import joblib

import os

import re

### (2) 데이터 불러오기

* 구글 드라이브 연결

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/KT_AIVLE/2023.10.30_미니프로젝트6차

/content/drive/MyDrive/KT_AIVLE/2023.10.30_미니프로젝트6차


In [ ]:
data = joblib.load('data')
data

,intent,Q,A,type
0,1,떨어뜨려서 핸드폰 액정 나갔어,as 맡기세요.,0
1,1,액정 나갔어,as 맡기세요.,0
2,1,핸드폰 떨어뜨려서 고장 났나봐,as 맡기세요.,0
3,1,노트북이 작동이 안되네,AS센터에 맡겨보세요.,0
4,1,노트북 키보드가 안먹히네,AS센터에 맡겨보세요.,0
...,...,...,...,...
787,53,KT 에이블스쿨만의 장점이 무엇인지 상세히 설명해주세요,KT 에이블스쿨에서는 이론/실습 교육에 나아가 기업 실전형 프로젝트를 중심으로 실무...,1
788,53,다른 교육과의 차이점은 무엇인가요?,KT 에이블스쿨에서는 이론/실습 교육에 나아가 기업 실전형 프로젝트를 중심으로 실무...,1
789,53,다른 교육과 차이점이 있나요?,KT 에이블스쿨에서는 이론/실습 교육에 나아가 기업 실전형 프로젝트를 중심으로 실무...,1
790,53,다른 국비 지원 교육보다 특별한 점이 있는지 궁금해요,KT 에이블스쿨에서는 이론/실습 교육에 나아가 기업 실전형 프로젝트를 중심으로 실무...,1


### (1) 학습/평가 데이터 분리
   * test 데이터 : intent마다 무작위로 질문 2개를 뽑아 test 데이터로 분리
   * train 데이터 : 나머지 데이터

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.3, shuffle=True, stratify=data['intent'])

In [ ]:
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [ ]:
train

,intent,Q,A,type
0,32,아르바이트를 하고 있는 경우 교육을 수강할 수 없나요?,"KT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가...",1
1,29,유기견 입양하려고,좋은 생각입니다.,0
2,8,대출 갚으면 한 평생 다 갈듯,너무 무리하면 지쳐요.,0
3,32,실업급여 수령 중에도 지원할 수 있나요?,"KT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가...",1
4,46,출석은 핸드폰으로 해도 되나요?,"교육기간 동안 노트북을 무료로 지원하며, 교육에 필요한 필수 프로그램이 설치되어 있...",1
...,...,...,...,...
903,29,책 좀 읽어야지,좋은 생각이에요.,0
904,32,교육 시작일 이후에 퇴사하는 경우 지원할 수 있나요?,"KT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가...",1
905,47,"교육기간 동안의 테스트 및 시험에서는 정해진 시간 안에 문제를 풀어야 하나요, 아니...","KT 에이블스쿨은 자기주도적인 학습을 장려하고 있으며, 교과목 종료 시 셀프테스트를...",1
906,8,생리통,힘들겠어요.,0


In [ ]:
test

,intent,Q,A,type
0,43,교육 장소를 변경하고 싶습니다.,"KT 에이블스쿨은 전국 어디서나 참여할 수 있도록, 온/오프라인을 병행하여 탄력적으...",1
1,40,코딩을 몰라도 괜찮나요?,비전공자도 SW개발에 관심과 열정이 있으신 분들은 충분히 따라 갈 수 있습니다. 이...,1
2,53,다른 국비지원 교육과 뭐가 다른가요?,KT 에이블스쿨에서는 이론/실습 교육에 나아가 기업 실전형 프로젝트를 중심으로 실무...,1
3,8,이 회사는 나만 일하는 것 같아,직장 스트레스 심하겠네요.,0
4,39,휴학생 졸업증명서를 대체 할 수 있는 서류가 있나요?,학교 사정으로 졸업예정증명서 발급이 불가능한 경우 수료증명서 또는 재학증명서를 제출...,1
...,...,...,...,...
385,45,교육 수강 중인데 다른 교육을 동시에 수강하고 싶은 경우 어떻게 해야 하나요?,"KT 에이블스쿨은 풀타임(09:00~18:00)으로 교육이 진행되며, 정해진 시간에...",1
386,36,면접 진행방식이 궁금합니다.,KT 에이블스쿨은 별도의 면접 과정이 없습니다,1
387,32,지원 가능한 미취업자의 기준을 자세히 알려주세요.,"KT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가...",1
388,40,미리 공부하면 좋은 것이 있나요?,비전공자도 SW개발에 관심과 열정이 있으신 분들은 충분히 따라 갈 수 있습니다. ...,1


In [ ]:
joblib.dump(train, 'train')
joblib.dump(test, 'test')

['test']

## 2.데이터 전처리

* 세부요구사항
    * 다음 항목에 해당되는 전처리 함수를 각각 생성하여 하나로 합쳐봅시다.
        * 특수문자 제거 : 정규 표현식을 활용합니다.
        * 형태소 분석기 사용 : konlpy(mecab, kkma, Okt, etc), soynlp, khaiii 등 여러 형태소 분석기를 활용합니다.
    * 문장길이를 조절하여 모델 input으로 활용하기
    * joblib.dump를 이용하여 전처리 결과 데이터를 저장합니다.

### (1) 맞춤법 검사하기
   * py-hanspell 설치 후 맞춤법 검사 함수 생성  
       🔗 https://github.com/ssut/py-hanspell

In [ ]:
!git clone https://github.com/ssut/py-hanspell.git

Cloning into 'py-hanspell'...
remote: Enumerating objects: 110, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 110 (delta 16), reused 11 (delta 9), pack-reused 88
Receiving objects: 100% (110/110), 22.27 KiB | 400.00 KiB/s, done.
Resolving deltas: 100% (55/55), done.


In [ ]:
%cd py-hanspell

/content/drive/MyDrive/KT_AIVLE/2023.10.30_미니프로젝트6차/py-hanspell


###setup.py 파일 설치 전 spell_checker.py 파일 수정


---


> 🚨 에러 발생 🚨   
> KeyError: 'result'

> 💡 hanspell 패키지는 내부적으로 NAVER 맞춤법 검사 API를 사용하여 한글 맞춤법 검사를 수행합니다. 따라서 API의 업데이트에 따라 추가적인 인증이 필요하게 되었습니다.

> Pull Request 52번 참고✨  
  (hanspell/spell_checker.py 파일 수정)  
  🔗 https://github.com/ssut/py-hanspell/pull/52  
> 🪄 토큰 관리 기능 추가  
> 🪄 API 요청 방식 변경  
> 🪄네이버 맞춤법 검사 페이지 크롤링 ➡️ 토큰 추출
> + 🚨 추가 에러 발생 🚨  
> UnboundLocalError: local variable 'TOKEN' referenced before assignment  
> 🪄 33줄에 TOKEN = None 추가 작성 필요  

###⚙️ 특정 Pull Request 적용하기(코랩)


---


방법 1. 복사/붙여넣기를 통해 변경사항 적용  
방법 2. 코드를 통해 변경사항 적용 🪄  
🔗 https://velog.io/@codnjs042/코랩Colab-환경에서-GitHub-API를-통해-Pull-Request-적용하기

In [ ]:
import requests

# 특정 Pull Request 적용 함수
def apply_pull_request(pull_request_url):

    # Pull Request의 변경된 파일 가져오기
    response = requests.get(pull_request_url)
    changed_files = response.json()

    # 파일 이름과 내용을 저장할 리스트
    files_data = []

    # 각 파일에 대한 변경 사항 가져오기
    for file in changed_files:
        file_name = file['filename']
        raw_url = file['raw_url']
        diff = file['patch']
        status = file['status']
        pre_name=''

        if status == "removed":
            original_content = requests.get(raw_url).text
            file_content = ''
            os.remove(file_name)

        elif status in ["added", "modified", "renamed"]:
            file_content = requests.get(raw_url).text

            if status == "added":
                mode = "w+"

            else:
                mode = "r+"

                # renamed : renamed and modified
                if status == "renamed":
                    pre_name = file["previous_filename"]
                    os.renamed(pre_name, file_name)

            with open(file_name, mode) as f:
                original_content = f.read()
                f.seek(0)
                f.write(file_content)
                f.seek(0)
                changed_content = f.read()

        # 파일 변경사항을 딕셔너리로 저장하여 리스트에 추가
        files_data.append({"filename": file_name, "original_content": original_content, "changed_content": changed_content, "diff": diff, "pre_name" : pre_name})

    return files_data

# 특정 Pull Request 적용
pull_request_url = "https://api.github.com/repos/ssut/py-hanspell/pulls/52/files"
apply_pull_request(pull_request_url)

[{'filename': 'hanspell/spell_checker.py',
  'original_content': '# -*- coding: utf-8 -*-\n"""\nPython용 한글 맞춤법 검사 모듈\n"""\n\nimport requests\nimport json\nimport time\nimport sys\nfrom collections import OrderedDict\nimport xml.etree.ElementTree as ET\n\nfrom . import __version__\nfrom .response import Checked\nfrom .constants import base_url\nfrom .constants import CheckResult\n\n_agent = requests.Session()\nPY3 = sys.version_info[0] == 3\n\n\ndef _remove_tags(text):\n    text = u\'<content>{}</content>\'.format(text).replace(\'<br>\',\'\')\n    if not PY3:\n        text = text.encode(\'utf-8\')\n\n    result = \'\'.join(ET.fromstring(text).itertext())\n\n    return result\n\n\ndef check(text):\n    """\n    매개변수로 입력받은 한글 문장의 맞춤법을 체크합니다.\n    """\n    if isinstance(text, list):\n        result = []\n        for item in text:\n            checked = check(item)\n            result.append(checked)\n        return result\n\n    # 최대 500자까지 가능.\n    if len(text) > 500:\n        return Chec

In [ ]:
!python setup.py install

/usr/local/lib/python3.10/dist-packages/setuptools/dist.py:755: SetuptoolsDeprecationWarning: Invalid dash-separated options
!!

        ********************************************************************************
        Usage of dash-separated 'description-file' will not be supported in future
        versions. Please use the underscore name 'description_file' instead.

        This deprecation is overdue, please update your project and remove deprecated
        calls to avoid build errors in the future.

        See https://setuptools.pypa.io/en/latest/userguide/declarative_config.html for details.
        ********************************************************************************

!!
  opt = self.warn_dash_deprecation(opt, section)
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
 

In [ ]:
from hanspell import spell_checker

def spell_check(original_sent):
    result = spell_checker.check(original_sent)

    return result.checked

In [ ]:
train['check'] = train['Q'].apply(lambda x : spell_check(x))
test['check'] = test['Q'].apply(lambda x : spell_check(x))

In [ ]:
train['check']

0                         아르바이트를 하고 있는 경우 교육을 수강할 수 없나요?
1                                              유기견 입양하려고
2                                       대출 갚으면 한 평생 다 갈듯
3                                 실업급여 수령 중에도 지원할 수 있나요?
4                                      출석은 핸드폰으로 해도 되나요?
                             ...                        
903                                             책 좀 읽어야지
904                        교육 시작일 이후에 퇴사하는 경우 지원할 수 있나요?
905    교육기간 동안의 테스트 및 시험에서는 정해진 시간 안에 문제를 풀어야 하나요, 아니...
906                                                  생리통
907                                       프리랜서는 미취업자인가요?
Name: check, Length: 908, dtype: object

### (3) K-TACC
   * 문맥을 고려한 한국어 텍스트 데이터 증강  
   (Korean Text Augmentation Considering Context, K-TACC)  
       🔗 https://github.com/kyle-bong/K-TACC  
       🚨 주의 사항 - ratio 값에 따라 무한 로딩 발생. 자세한 원인 찾는 중.

In [ ]:
%cd ..

/content/drive/MyDrive/KT_AIVLE/2023.10.30_미니프로젝트6차


In [ ]:
!git clone https://github.com/kyle-bong/K-TACC.git

Cloning into 'K-TACC'...
remote: Enumerating objects: 476, done.
remote: Counting objects: 100% (191/191), done.
remote: Compressing objects: 100% (137/137), done.
remote: Total 476 (delta 127), reused 84 (delta 54), pack-reused 285
Receiving objects: 100% (476/476), 35.70 MiB | 12.68 MiB/s, done.
Resolving deltas: 100% (326/326), done.
Updating files: 100% (35/35), done.


In [5]:
%cd K-TACC

/content/drive/MyDrive/KT_AIVLE/2023.10.30_미니프로젝트6차/K-TACC


In [6]:
from BERT_augmentation import BERT_Augmentation
augmenter = BERT_Augmentation()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/463 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/149M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

In [8]:
# 무한 로딩 방지

import random
import signal

class TimeoutError(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutError("Function call timed out")

def bert_aug(sentence, num):
    aug_sent = set()

    # Register the signal handler
    signal.signal(signal.SIGALRM, timeout_handler)

    for x in range(num):
        try:
            ratio = round(random.uniform(0.15, 0.7), 2)
            print(ratio)

            # 랜덤 마스킹 치환 적용
            signal.alarm(3)  # Set the timeout for 5 seconds
            masked_sentence = augmenter.random_masking_replacement(sentence, ratio)
            aug_sent.add(masked_sentence)
            signal.alarm(0)  # Cancel the timeout
            print('치환', aug_sent)

            # 랜덤 마스킹 삽입 적용
            signal.alarm(3)  # Set the timeout for 5 seconds
            masked_inserted_sentence = augmenter.random_masking_insertion(sentence, ratio)
            aug_sent.add(masked_inserted_sentence)
            signal.alarm(0)  # Cancel the timeout
            print('삽입', aug_sent)
        except TimeoutError:
            print("Function call timed out, skipping this iteration...")
            continue
        except Exception as e:
            print(f"Error occurred: {e}")
            continue

    return aug_sent



In [9]:
eda = train['check'].apply(lambda x: bert_aug(x, 20))

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
0.35
치환 {'졸업이 조금씩 미뤄졌는데 괜찮나요?', '졸업이 미뤄졌는데 그래도 괜찮나요?', '졸업이 미뤄졌는데 괜찮나요?', '졸업이 조금 미뤄졌는데, 괜찮나요?'}
삽입 {'졸업이 조금 미뤄졌는데, 괜찮나요?', '졸업이 조금씩 미뤄졌는데 괜찮나요?', '졸업이 미뤄졌는데 그래도 괜찮나요?', '졸업이 조금 미뤄졌는데 괜찮나요?', '졸업이 미뤄졌는데 괜찮나요?'}
0.22
치환 {'졸업이 조금 미뤄졌는데, 괜찮나요?', '졸업이 조금씩 미뤄졌는데 괜찮나요?', '졸업이 미뤄졌는데 그래도 괜찮나요?', '졸업이 조금 미뤄졌는데 괜찮나요?', '졸업이 미뤄졌는데 괜찮나요?'}
삽입 {'졸업이 조금 미뤄졌는데, 괜찮나요?', '졸업이 조금씩 미뤄졌는데 괜찮나요?', '졸업이 미뤄졌는데 그래도 괜찮나요?', '졸업이 조금 미뤄졌는데 괜찮나요?', '졸업이 미뤄졌는데 괜찮나요?'}
0.46
치환 {'졸업이 조금 미뤄졌는데, 괜찮나요?', '졸업이 조금씩 미뤄졌는데 괜찮나요?', '졸업이 미뤄졌는데 그래도 괜찮나요?', '졸업이 조금 미뤄졌는데 괜찮나요?', '졸업이 미뤄졌는데 괜찮나요?'}
삽입 {'졸업이 조금 미뤄졌는데, 괜찮나요?', '졸업이 조금씩 미뤄졌는데 괜찮나요?', '졸업이 미뤄졌는데 그래도 괜찮나요?', '졸업이 조금 미뤄졌는데 괜찮나요?', '졸업이 미뤄졌는데 괜찮나요?'}
0.66
치환 {'졸업이 조금 미뤄졌는데, 괜찮나요?', '졸업이 조금씩 미뤄졌는데 괜찮나요?', '졸업이 미뤄졌는데 그래도 괜찮나요?', '졸업이 조금 미뤄졌는데 괜찮나요?', '졸업이 미뤄졌는데 괜찮나요?'}
삽입 {'졸업이 조금 미뤄졌는데, 괜찮나요?', '졸업이 조금씩 미뤄졌는데 괜찮나요?', '졸업이라 조금 미뤄졌는데 괜찮나요?', '졸업이 미뤄졌는데 그래도 괜찮나요?', '졸업이 조금 미뤄졌는데 괜찮나요?', '졸업이 미뤄졌는데 괜찮나요?'}
0.38
치환 {'졸업이 조금 미뤄졌는데, 괜찮나요?',

In [10]:
eda

0      {아르바이트를 하고 다닐 때는 수강할 수 없나요?, 아르바이트를 하고도 교육을 시킬...
1                 {그리고 유기견 입양하려고, 유기견 입양하려고, 유기견을 입양하려고}
2      {대출 갚으면 한 평생을 거의 다 살아 갈듯, 학자금 대출 갚으면 한 평생 다 살아...
3      {실업급여 수령자 중에도 지원할 수 있나요?, 혹시 실업급여 수령 중에도 지원할 할...
4      {출석은 그냥 그냥 핸드폰으로 해도 되나요?, 출석은 핸드폰으로만 해도 되나요?, ...
                             ...                        
903    {이 책을 좀 읽어야지, 이제 책 좀 읽어야지, 자 이제 책 좀 읽어야지, 이제 이...
904    {혹시 교육 시작일 이후에 퇴사하는 분의 분의 경우 지원할 수 있나요?, 교육 시작...
905    {교육기간 동안의 테스트 및 또는 정해진 시간 안에 문제를 풀어야 하나요, 아니면 ...
906                                        {생리통, 만성 생리통}
907    {프리랜서는? 미취업자인가요?, 그럼 프리랜서는 미취업자인가요?, 프리랜서는 미취업...
Name: check, Length: 908, dtype: object

In [11]:
eda.apply(len).sum()

15859

### 🚨 메모리 이슈 주의!
데이터 처리량 과다 시 메모리 이슈로 인해 일부 데이터가 생략될 수 있다.  
💡 해결 방법 - 데이터를 나누어 처리


```
def process_data(data, eda, num_partitions):
    new_rows = []

    # 데이터를 num_partitions 등분으로 나누기
    partition_size = len(data) // num_partitions
    for index in range(0, len(data), partition_size):
        for idx in range(index, min(index + partition_size, len(data))):
            row = data.iloc[idx]
            eda_sentences = eda.iloc[idx]
            for sentence in eda_sentences:
                new_rows.append({'intent': row['intent'], 'Q': sentence, 'A': row['A'], 'type': row['type']})
    
    return pd.DataFrame(new_rows)

# 사용 예시
num_partitions = 3  # 데이터를 세 등분으로 나눔
eda_data = process_data(train, eda, num_partitions)
```




In [14]:
# 결과 데이터프레임 초기화
new_rows = []

# 기존 데이터프레임을 이용하여 새로운 행 생성
for index, row in train.iterrows():
    for sentence in eda.iloc[index]:
        new_rows.append({'intent': row['intent'], 'Q': sentence, 'A': row['A'], 'type': row['type']})

# 새로운 데이터프레임 생성
eda_data = pd.DataFrame(new_rows)

# 결과 출력
eda_data

,intent,Q,A,type
0,32,아르바이트를 하고 다닐 때는 수강할 수 없나요?,"KT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가...",1
1,32,아르바이트를 하고도 교육을 시킬 수 없나요?,"KT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가...",1
2,32,아르바이트를 많이 하고만 있는 분의 경우 교육을 수강할 할 수 없나요?,"KT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가...",1
3,32,아르바이트를 하고 있는 분의 경우 교육을 제대로 제대로 받아 수강할 할 수 없나요?,"KT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가...",1
4,32,아르바이트를 하고만 남아 있는 분의 경우 교육을 수강할 수 없나요?,"KT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가...",1
...,...,...,...,...
15854,8,생리통,힘들겠어요.,0
15855,8,만성 생리통,힘들겠어요.,0
15856,32,프리랜서는? 미취업자인가요?,"KT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가...",1
15857,32,그럼 프리랜서는 미취업자인가요?,"KT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가...",1


In [15]:
eda_data.drop_duplicates(subset=['Q'], inplace=True)
eda_data.reset_index(drop=True, inplace=True)
eda_data

,intent,Q,A,type
0,32,아르바이트를 하고 다닐 때는 수강할 수 없나요?,"KT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가...",1
1,32,아르바이트를 하고도 교육을 시킬 수 없나요?,"KT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가...",1
2,32,아르바이트를 많이 하고만 있는 분의 경우 교육을 수강할 할 수 없나요?,"KT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가...",1
3,32,아르바이트를 하고 있는 분의 경우 교육을 제대로 제대로 받아 수강할 할 수 없나요?,"KT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가...",1
4,32,아르바이트를 하고만 남아 있는 분의 경우 교육을 수강할 수 없나요?,"KT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가...",1
...,...,...,...,...
15715,8,생리통,힘들겠어요.,0
15716,8,만성 생리통,힘들겠어요.,0
15717,32,프리랜서는? 미취업자인가요?,"KT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가...",1
15718,32,그럼 프리랜서는 미취업자인가요?,"KT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가...",1


### (5) 전처리 완료된 학습/평가 데이터를 파일로 저장하기
- 총 2개의 파일을 저장합니다.
    * train 셋
    * test 셋
- 저장방법
    * joblib 라이브러리를 이용하여 저장합니다.

In [ ]:
%cd ..

/content/drive/MyDrive/KT_AIVLE/2023.10.30_미니프로젝트6차


In [16]:
joblib.dump(train, 'train')
joblib.dump(eda_data, 'eda_data')
joblib.dump(test, 'test')

['test']